In [25]:
import spacy
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
import nltk
import torch
from scipy.spatial.distance import cosine
import nltk
from nltk.corpus import stopwords
from scipy import spatial
from nltk.corpus import wordnet as wn
stop_words = set(stopwords.words('english'))
nlp = spacy.load('en_core_web_sm')
#nltk.download('wordnet')

In [26]:

from pattern.en import conjugate, lemma, lexeme, PRESENT,PAST,FUTURE,PARTICIPLE, SG,tenses

def get_verb_tense(verb):
    if(PAST in tenses(verb)):
        return PAST
    if(PRESENT in tenses(verb)):
        return PRESENT
    if(PRESENT in tenses(verb)):
        return PRESENT
    if('progressive' in tenses(verb)[0][4]):
        return PARTICIPLE


# 2
def my_conjugate(verb1, verb2_stem):
    
    verb1_stem = lemma(verb1)
    if(verb1_stem == verb1):
        return verb2_stem
    verb1_tense = get_verb_tense(verb1)
    verb2 = conjugate(verb=verb2_stem,tense=verb1_tense,number=SG)
    return verb2
    
verb1 = "like"
verb2_stem = "care"
my_conjugate(verb1, verb2_stem)

'care'

In [27]:
'''
this class is used to recognize the named entities in the sentence
and encode them in a way that ensure they won't be replaced when paraphrasing
'''
class NerEncoder():
    def __init__(self):
        self.ner_dict = {}
        self.label_index = 0
        
        
    def translate(self, label, word):
        self.label_index += 1
        if label == 'NORP':
            self.ner_dict["Group" + str(self.label_index)] = word
            return "Group" + str(self.label_index)
        if label == 'FAC':
            self.ner_dict["Place" + str(self.label_index)] = word
            return "Place" + str(self.label_index)
        if label == 'ORG':
            self.ner_dict["Organization" + str(self.label_index)] = word
            return "Organization" + str(self.label_index)
        if label == 'GPE':
            self.ner_dict["State" + str(self.label_index)] = word
            return "State" + str(self.label_index)
        if label == 'LOC':
            self.ner_dict["Location" + str(self.label_index)] = word
            return "Location" + str(self.label_index)
        if label == 'PERCENT':
            self.ner_dict["Percentage" + str(self.label_index)] = word
            return "Percentage" + str(self.label_index)
        if label == 'ORDINAL':
            self.ner_dict["Number" + str(self.label_index)] = word
            return "Number" + str(self.label_index)
        if label == 'CARDINAL':
            self.ner_dict["first" + str(self.label_index)] = word
            return "first" + str(self.label_index)
        else:
            self.ner_dict[label + str(self.label_index)] = word
            return label + str(self.label_index)
        
    
    def encode(self, doc):
        res = ""
        b = False
        for i, token in enumerate(doc):
            for ent in doc.ents:
                if i == ent.start:
                    b = True
                    res += self.translate(ent.label_, ent.text) + " "
                if i == ent.end:
                    b = False
            if b==False:
                res += token.text +" "
        return res
    def decode(self, doc):
        res = ""
        for token in doc:
            if token.text in self.ner_dict:
                res += self.ner_dict[token.text] + " "
            else:
                res += token.text + " "
        return res

In [28]:
#example of using NerEncoder
doc = nlp(u'Tony Mark Smith loves Massachusetts Institute of Technology, he lives in the United States of America')
encoder = NerEncoder()
print(encoder.encode(doc))
print(encoder.decode(nlp(encoder.encode(doc))))

PERSON1 loves Organization2 , he lives in State3 
Tony Mark Smith loves Massachusetts Institute of Technology , he lives in the United States of America 


In [29]:
# Loading the pre-trained BERT model
###################################
# Embeddings will be derived from
# the outputs of this model
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True,
                                  )

# Setting up the tokenizer
###################################
# This is the same tokenizer that
# was used in the model to generate 
# embeddings to ensure consistency
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
def bert_text_preparation(text, tokenizer):
    """Preparing the input for BERT
    
    Takes a string argument and performs
    pre-processing like adding special tokens,
    tokenization, tokens to ids, and tokens to
    segment ids. All tokens are mapped to seg-
    ment id = 1.
    
    Args:
        text (str): Text to be converted
        tokenizer (obj): Tokenizer object
            to convert text into BERT-re-
            adable tokens and ids
        
    Returns:
        list: List of BERT-readable tokens
        obj: Torch tensor with token ids
        obj: Torch tensor segment ids
    
    
    """
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1]*len(indexed_tokens)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    return tokenized_text, tokens_tensor, segments_tensors
    
def get_bert_embeddings(tokens_tensor, segments_tensors, model):
    """Get embeddings from an embedding model
    
    Args:
        tokens_tensor (obj): Torch tensor size [n_tokens]
            with token ids for each token in text
        segments_tensors (obj): Torch tensor size [n_tokens]
            with segment ids for each token in text
        model (obj): Embedding model to generate embeddings
            from token and segment ids
    
    Returns:
        list: List of list of floats of size
            [n_tokens, n_embedding_dimensions]
            containing embeddings for each token
    
    """
    
    # Gradient calculation id disabled
    # Model is in inference mode
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        # Removing the first hidden state
        # The first state is the input state
        hidden_states = outputs[2][1:]

    # Getting embeddings from the final BERT layer
    token_embeddings = hidden_states[-1]
    # Collapsing the tensor into 1-dimension
    token_embeddings = torch.squeeze(token_embeddings, dim=0)
    # Converting torchtensors to lists
    list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]

    return list_token_embeddings

In [31]:
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

def pos_extractor(pos):
    if pos == "NOUN":
        return "n"
    if pos == "VERB":
        return "v"
    if pos == "ADJ":
        return "a"
    if pos == "ADV":
        return "r"
    else:
        return "o"
    
    
def get_index(sent, words):
    ps = PorterStemmer()
    words = [ps.stem(w) for w in words]
    for i, word in enumerate(sent.split()):
        if ps.stem(word) in words:
            return i

In [32]:
def all_subsets(lst1, lst2):
    res = []
    for item1 in lst1:
        for item2 in lst2:
            f = item1.copy()
            f.append(item2)
            res.append(f)
    return res


def generate_paraphrase(to_be_replaced, encoder, max_num):
    """
    this functions a list of lists, NER encoder, and max_num
    the first parameter has a list for every word in the sentence containing the potential replacements with their similarity
    as tuple, along with the original word with similarity of 1.
    the second parameter is the NER encoder used, in order to decode the result.
    the last one is the number of paraphrases wanted
    """
    subsets = []
    for i in range(len(to_be_replaced[0])):
        subsets.append([i])
    for i, replace in enumerate(to_be_replaced):
        if i == 0:
            continue
        a = [i for i in range(len(replace))]
        subsets = all_subsets(subsets, a)
    res = []
    for subset in subsets:
        text = ""
        score = 1.0
        for i, word_index in enumerate(subset):
            text += to_be_replaced[i][word_index][0]
            text += " "
            score*=to_be_replaced[i][word_index][1]
        sent = tuple((encoder.decode(nlp(text)), score))
        res.append(sent)
    sorted_list = sorted(res, key=lambda x: x[1], reverse = True) 
    return sorted_list[:min(max_num, len(sorted_list))]

In [33]:
def paraphrase(doc):
    """
    this function takes the sentence and outputs its paraphrases
    """
    to_be_replaced = []
    doc = nlp(encoder.encode(doc))
    tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(doc.text, tokenizer)
    list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)
    for i, token in enumerate(doc):
        if pos_extractor(token.pos_) == "o":
            to_be_replaced.append([tuple((token.text, 1.0))])
            continue
        if tokenized_text.count(token.text) > 0:
            word_index = tokenized_text.index(token.text)
            word_embedding = list_token_embeddings[word_index]
        else:
            to_be_replaced.append([tuple((token.text, 1.0))])
            continue
        synsets = wn.synsets(token.text, pos = pos_extractor(token.pos_))
        to_be_appended = [tuple((token.text, 1.0))]
        new_words = set(token.text)
        new_words.add(token.lemma_)
        for synset in synsets:
            names = [x.name() for x in synset.lemmas()]
            for name in names:
                context = doc.text.split(' ')
                context[i] = name
                tokenized_text_example, tokens_tensor_example, segments_tensors_example = bert_text_preparation(str(context), tokenizer)
                list_token_embeddings_example = get_bert_embeddings(tokens_tensor_example, segments_tensors_example, model)
                if tokenized_text_example.count(name) > 0:
                    word_index_example = tokenized_text_example.index(name)
                    word_embedding_example = list_token_embeddings_example[word_index_example]
                    if name not in new_words:
                        new_words.add(name)
                        if pos_extractor(token.pos_) == "v":
                                to_be_appended.append(tuple((my_conjugate(token.text, name), 1 - cosine(word_embedding, word_embedding_example))))
                        else:
                                to_be_appended.append(tuple((name, 1 - cosine(word_embedding, word_embedding_example))))

        to_be_replaced.append(to_be_appended)
    result = generate_paraphrase(to_be_replaced, encoder, 10)
    for res in result:
        print(res)

In [35]:
#example1
doc = nlp("he achieved his goal")
paraphrase(doc)

('he achieved his goal ', 1.0)
('he achieved his destination ', 0.5151607832447769)
('he accomplished his goal ', 0.5039548547197482)
('he achieved his finish ', 0.4545435950979153)
('he attained his goal ', 0.4398768661175332)
('he achieved his end ', 0.43298626937261275)
('he reached his goal ', 0.4313708732398621)
('he accomplished his destination ', 0.2596177776774332)
('he accomplished his finish ', 0.22906945143136195)
('he attained his destination ', 0.22660731088036626)


In [36]:
#example2
doc = nlp("messi won the world cup to prove he is the best and end the debate")
paraphrase(doc)

('messi won the world cup to prove he is the best and end the debate ', 1.0)
('messi won the world cup to prove he is the best and end the debate ', 0.6888724530837383)
('messi won the world cup to prove he is the best and end the argument ', 0.5631689713651935)
('messi won the world cup to test he is the best and end the debate ', 0.5400497893885152)
('messi won the world cup to demonstrate he is the best and end the debate ', 0.5261544497239785)
('messi won the world cup to show he is the best and end the debate ', 0.5243720718479813)
('messi won the world cup to prove he is the best and stop the debate ', 0.5237512140363605)
('messi won the world cup to prove he is the expert and end the debate ', 0.5012637257048709)
('messi won the world cup to try he is the best and end the debate ', 0.49333191724091763)
('messi won the world cup to establish he is the best and end the debate ', 0.49215984021086)
